In [1]:
## This file implements neural networks and logistic regression on p0040presabs_quant.
## For fully-connected neural networks, the accuracy is 74.03%, and 77.92% after improvement.
## We further implement the over-sampling method and the combination of over- and under-sampling method, in case of imbalanced data.
## For fully-connected neural networks, the accuracy is 98.39% after regularization for combination data
## and 84.85% for over-sampling data.
## For logistic regression, the accuracies are 77.92% without balanced methods, 98.39% for combination data and 86.87% for over-sampling data.
## For random forest, the accuracy is 77.92% without balanced methods, 98.39% for combination, and 87.88% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 83.70% without balanced methods, 97.24% for combination data, and 83.96% for over-sampling.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [4]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p0040presabs_quant.csv')
df.shape

(255, 373)

In [5]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [6]:
df['pheno']

0      0.814000
1      0.365125
2      0.803875
3      1.088125
4      1.081750
5      1.112875
6      0.817875
7      0.398250
8      0.569500
9      0.526125
10     0.351625
11     0.814875
12     0.590875
13     0.373875
14     0.922625
15     0.955750
16     0.383500
17     0.379625
18     0.689625
19     0.186750
20     0.351750
21     0.364250
22     0.306125
23     1.112250
24     0.963000
25     0.159500
26     0.824750
27     0.196000
28     0.590875
29     0.830375
         ...   
225    0.715500
226    0.469000
227    0.670400
228    0.746000
229    0.969000
230    0.577125
231    0.627375
232    0.930500
233    0.478875
234    0.602500
235    0.900125
236    0.646875
237    0.847125
238    0.461500
239    0.258375
240    0.515875
241    0.466875
242    0.275500
243    0.542125
244    0.486625
245    0.765375
246    0.845625
247    0.530875
248    0.894625
249    0.457250
250    0.868000
251    0.550500
252    0.685125
253    0.939500
254    0.926500
Name: pheno, Length: 255

In [7]:
df.head()

,id,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGGACAGGCAGTGAAACAGAATATAATTATTTGTA,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,...,hxlA_2,group_8822,group_10124,group_1648,group_426,group_8122,group_8213,group_8913,group_9905,pheno
0,107,1,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0.814000
1,109,0,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0.365125
2,115,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0.803875
3,120335,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1.088125
4,120337,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1.081750


In [8]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [9]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [10]:
df['pheno']
df['pheno'].value_counts()

1    165
0     90
Name: pheno, dtype: int64

In [11]:
df.shape

(255, 373)

In [12]:
df_clean = df.drop(columns=['id'])

In [13]:
df_clean.shape

(255, 372)

In [14]:
df_clean.head()

,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGGACAGGCAGTGAAACAGAATATAATTATTTGTA,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,TTTCGCTGTTGCAACATCTTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACT,...,hxlA_2,group_8822,group_10124,group_1648,group_426,group_8122,group_8213,group_8913,group_9905,pheno
0,1,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
3,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
4,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [15]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 371) (255,)


In [16]:
# split into train, test data (original)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y)

In [17]:
############# Fully-Connected Neural Network ################

In [18]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

Using TensorFlow backend.


In [19]:
#### neural network
model1 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [20]:
model1.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [21]:
model1.fit(X_train, y_train,
          batch_size=32, epochs=1000,
          validation_data=(X_test, y_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 767us/step - loss: 0.6608 - accuracy: 0.6461 - val_loss: 0.6247 - val_accuracy: 0.6753
Epoch 2/1000
178/178 [==============================] - 0s 593us/step - loss: 0.5679 - accuracy: 0.6910 - val_loss: 0.5954 - val_accuracy: 0.6883
Epoch 3/1000
178/178 [==============================] - 0s 372us/step - loss: 0.5203 - accuracy: 0.7640 - val_loss: 0.5812 - val_accuracy: 0.7013
Epoch 4/1000
178/178 [==============================] - 0s 602us/step - loss: 0.4921 - accuracy: 0.7528 - val_loss: 0.5745 - val_accuracy: 0.6883
Epoch 5/1000
178/178 [==============================] - 0s 152us/step - loss: 0.4744 - accuracy: 0.7472 - val_loss: 0.5695 - val_accuracy: 0.6883
Epoch 6/1000
178/178 [==============================] - 0s 183us/step - loss: 0.4637 - accuracy: 0.7640 - val_loss: 0.5675 - val_accuracy: 0.6883
Epoch 7/1000
178/178 [==============================] - 0s 141us/step - loss: 0

178/178 [==============================] - 0s 209us/step - loss: 0.3514 - accuracy: 0.8764 - val_loss: 0.5446 - val_accuracy: 0.7273
Epoch 57/1000
178/178 [==============================] - 0s 199us/step - loss: 0.3492 - accuracy: 0.8764 - val_loss: 0.5445 - val_accuracy: 0.7273
Epoch 58/1000
178/178 [==============================] - 0s 157us/step - loss: 0.3502 - accuracy: 0.8764 - val_loss: 0.5451 - val_accuracy: 0.7273
Epoch 59/1000
178/178 [==============================] - 0s 158us/step - loss: 0.3471 - accuracy: 0.8708 - val_loss: 0.5462 - val_accuracy: 0.7403
Epoch 60/1000
178/178 [==============================] - 0s 179us/step - loss: 0.3470 - accuracy: 0.8708 - val_loss: 0.5479 - val_accuracy: 0.7403
Epoch 61/1000
178/178 [==============================] - 0s 174us/step - loss: 0.3472 - accuracy: 0.8708 - val_loss: 0.5483 - val_accuracy: 0.7273
Epoch 62/1000
178/178 [==============================] - 0s 140us/step - loss: 0.3450 - accuracy: 0.8764 - val_loss: 0.5488 - val_ac

178/178 [==============================] - 0s 124us/step - loss: 0.3016 - accuracy: 0.8933 - val_loss: 0.5522 - val_accuracy: 0.7273
Epoch 112/1000
178/178 [==============================] - 0s 112us/step - loss: 0.3026 - accuracy: 0.8933 - val_loss: 0.5524 - val_accuracy: 0.7273
Epoch 113/1000
178/178 [==============================] - 0s 115us/step - loss: 0.2997 - accuracy: 0.8820 - val_loss: 0.5532 - val_accuracy: 0.7403
Epoch 114/1000
178/178 [==============================] - 0s 104us/step - loss: 0.3010 - accuracy: 0.8989 - val_loss: 0.5507 - val_accuracy: 0.7403
Epoch 115/1000
178/178 [==============================] - 0s 110us/step - loss: 0.2984 - accuracy: 0.8933 - val_loss: 0.5522 - val_accuracy: 0.7403
Epoch 116/1000
178/178 [==============================] - 0s 107us/step - loss: 0.2973 - accuracy: 0.8989 - val_loss: 0.5515 - val_accuracy: 0.7403
Epoch 117/1000
178/178 [==============================] - 0s 114us/step - loss: 0.2956 - accuracy: 0.8933 - val_loss: 0.5531 - 

178/178 [==============================] - 0s 112us/step - loss: 0.2646 - accuracy: 0.9045 - val_loss: 0.5640 - val_accuracy: 0.7532
Epoch 167/1000
178/178 [==============================] - 0s 98us/step - loss: 0.2632 - accuracy: 0.9045 - val_loss: 0.5639 - val_accuracy: 0.7532
Epoch 168/1000
178/178 [==============================] - 0s 96us/step - loss: 0.2610 - accuracy: 0.9045 - val_loss: 0.5645 - val_accuracy: 0.7662
Epoch 169/1000
178/178 [==============================] - 0s 125us/step - loss: 0.2610 - accuracy: 0.9045 - val_loss: 0.5665 - val_accuracy: 0.7532
Epoch 170/1000
178/178 [==============================] - 0s 101us/step - loss: 0.2634 - accuracy: 0.9045 - val_loss: 0.5654 - val_accuracy: 0.7532
Epoch 171/1000
178/178 [==============================] - 0s 98us/step - loss: 0.2579 - accuracy: 0.9101 - val_loss: 0.5651 - val_accuracy: 0.7532
Epoch 172/1000
178/178 [==============================] - 0s 137us/step - loss: 0.2589 - accuracy: 0.9045 - val_loss: 0.5645 - val

178/178 [==============================] - 0s 111us/step - loss: 0.2303 - accuracy: 0.9101 - val_loss: 0.5773 - val_accuracy: 0.7532
Epoch 222/1000
178/178 [==============================] - 0s 134us/step - loss: 0.2290 - accuracy: 0.9157 - val_loss: 0.5757 - val_accuracy: 0.7792
Epoch 223/1000
178/178 [==============================] - 0s 165us/step - loss: 0.2291 - accuracy: 0.9213 - val_loss: 0.5780 - val_accuracy: 0.7792
Epoch 224/1000
178/178 [==============================] - 0s 119us/step - loss: 0.2303 - accuracy: 0.9101 - val_loss: 0.5769 - val_accuracy: 0.7662
Epoch 225/1000
178/178 [==============================] - 0s 129us/step - loss: 0.2291 - accuracy: 0.9213 - val_loss: 0.5780 - val_accuracy: 0.7662
Epoch 226/1000
178/178 [==============================] - 0s 164us/step - loss: 0.2285 - accuracy: 0.9157 - val_loss: 0.5786 - val_accuracy: 0.7662
Epoch 227/1000
178/178 [==============================] - 0s 155us/step - loss: 0.2272 - accuracy: 0.9157 - val_loss: 0.5817 - 

...

178/178 [==============================] - 0s 202us/step - loss: 0.0923 - accuracy: 0.9719 - val_loss: 0.8052 - val_accuracy: 0.7532
Epoch 773/1000
178/178 [==============================] - 0s 213us/step - loss: 0.0903 - accuracy: 0.9719 - val_loss: 0.8026 - val_accuracy: 0.7532
Epoch 774/1000
178/178 [==============================] - 0s 202us/step - loss: 0.0912 - accuracy: 0.9719 - val_loss: 0.8041 - val_accuracy: 0.7532
Epoch 775/1000
178/178 [==============================] - 0s 183us/step - loss: 0.0890 - accuracy: 0.9719 - val_loss: 0.8069 - val_accuracy: 0.7532
Epoch 776/1000
178/178 [==============================] - 0s 156us/step - loss: 0.0891 - accuracy: 0.9719 - val_loss: 0.8034 - val_accuracy: 0.7532
Epoch 777/1000
178/178 [==============================] - 0s 145us/step - loss: 0.0902 - accuracy: 0.9719 - val_loss: 0.8039 - val_accuracy: 0.7662
Epoch 778/1000
178/178 [==============================] - 0s 139us/step - loss: 0.0894 - accuracy: 0.9775 - val_loss: 0.8096 - 

178/178 [==============================] - 0s 103us/step - loss: 0.0852 - accuracy: 0.9719 - val_loss: 0.8182 - val_accuracy: 0.7403
Epoch 828/1000
178/178 [==============================] - 0s 94us/step - loss: 0.0852 - accuracy: 0.9775 - val_loss: 0.8250 - val_accuracy: 0.7532
Epoch 829/1000
178/178 [==============================] - 0s 114us/step - loss: 0.0857 - accuracy: 0.9775 - val_loss: 0.8251 - val_accuracy: 0.7532
Epoch 830/1000
178/178 [==============================] - 0s 112us/step - loss: 0.0841 - accuracy: 0.9775 - val_loss: 0.8291 - val_accuracy: 0.7532
Epoch 831/1000
178/178 [==============================] - 0s 195us/step - loss: 0.0853 - accuracy: 0.9719 - val_loss: 0.8251 - val_accuracy: 0.7532
Epoch 832/1000
178/178 [==============================] - 0s 204us/step - loss: 0.0846 - accuracy: 0.9719 - val_loss: 0.8188 - val_accuracy: 0.7532
Epoch 833/1000
178/178 [==============================] - 0s 259us/step - loss: 0.0847 - accuracy: 0.9775 - val_loss: 0.8263 - v

178/178 [==============================] - 0s 171us/step - loss: 0.0804 - accuracy: 0.9775 - val_loss: 0.8397 - val_accuracy: 0.7532
Epoch 883/1000
178/178 [==============================] - 0s 165us/step - loss: 0.0798 - accuracy: 0.9775 - val_loss: 0.8435 - val_accuracy: 0.7532
Epoch 884/1000
178/178 [==============================] - 0s 189us/step - loss: 0.0811 - accuracy: 0.9775 - val_loss: 0.8422 - val_accuracy: 0.7532
Epoch 885/1000
178/178 [==============================] - 0s 166us/step - loss: 0.0794 - accuracy: 0.9719 - val_loss: 0.8414 - val_accuracy: 0.7532
Epoch 886/1000
178/178 [==============================] - 0s 172us/step - loss: 0.0794 - accuracy: 0.9775 - val_loss: 0.8457 - val_accuracy: 0.7532
Epoch 887/1000
178/178 [==============================] - 0s 213us/step - loss: 0.0809 - accuracy: 0.9775 - val_loss: 0.8464 - val_accuracy: 0.7532
Epoch 888/1000
178/178 [==============================] - 0s 181us/step - loss: 0.0794 - accuracy: 0.9775 - val_loss: 0.8480 - 

178/178 [==============================] - 0s 115us/step - loss: 0.0788 - accuracy: 0.9719 - val_loss: 0.8555 - val_accuracy: 0.7273
Epoch 938/1000
178/178 [==============================] - 0s 114us/step - loss: 0.0796 - accuracy: 0.9775 - val_loss: 0.8698 - val_accuracy: 0.7532
Epoch 939/1000
178/178 [==============================] - 0s 94us/step - loss: 0.0757 - accuracy: 0.9719 - val_loss: 0.8656 - val_accuracy: 0.7532
Epoch 940/1000
178/178 [==============================] - 0s 109us/step - loss: 0.0793 - accuracy: 0.9719 - val_loss: 0.8647 - val_accuracy: 0.7532
Epoch 941/1000
178/178 [==============================] - 0s 93us/step - loss: 0.0759 - accuracy: 0.9719 - val_loss: 0.8643 - val_accuracy: 0.7403
Epoch 942/1000
178/178 [==============================] - 0s 106us/step - loss: 0.0756 - accuracy: 0.9719 - val_loss: 0.8665 - val_accuracy: 0.7532
Epoch 943/1000
178/178 [==============================] - 0s 83us/step - loss: 0.0773 - accuracy: 0.9775 - val_loss: 0.8659 - val

178/178 [==============================] - 0s 206us/step - loss: 0.0741 - accuracy: 0.9719 - val_loss: 0.8738 - val_accuracy: 0.7273
Epoch 993/1000
178/178 [==============================] - 0s 188us/step - loss: 0.0757 - accuracy: 0.9831 - val_loss: 0.8792 - val_accuracy: 0.7403
Epoch 994/1000
178/178 [==============================] - 0s 205us/step - loss: 0.0778 - accuracy: 0.9719 - val_loss: 0.8837 - val_accuracy: 0.7532
Epoch 995/1000
178/178 [==============================] - 0s 161us/step - loss: 0.0736 - accuracy: 0.9775 - val_loss: 0.8916 - val_accuracy: 0.7532
Epoch 996/1000
178/178 [==============================] - 0s 168us/step - loss: 0.0749 - accuracy: 0.9719 - val_loss: 0.8864 - val_accuracy: 0.7532
Epoch 997/1000
178/178 [==============================] - 0s 169us/step - loss: 0.0741 - accuracy: 0.9719 - val_loss: 0.8915 - val_accuracy: 0.7532
Epoch 998/1000
178/178 [==============================] - 0s 179us/step - loss: 0.0736 - accuracy: 0.9719 - val_loss: 0.8871 - 

In [22]:
acc_test1 = model1.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test1*100))

77/77 [==============================] - 0s 115us/step
test accuracy: 74.03%


In [23]:
#### improve neural network
model2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [24]:
model2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [25]:
model2.fit(X_train, y_train,
          batch_size=64, epochs=1000,
          validation_data=(X_test, y_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 1ms/step - loss: 0.6683 - accuracy: 0.6404 - val_loss: 0.6513 - val_accuracy: 0.6494
Epoch 2/1000
178/178 [==============================] - 0s 162us/step - loss: 0.6529 - accuracy: 0.6461 - val_loss: 0.6406 - val_accuracy: 0.6623
Epoch 3/1000
178/178 [==============================] - 0s 138us/step - loss: 0.6372 - accuracy: 0.6461 - val_loss: 0.6305 - val_accuracy: 0.6494
Epoch 4/1000
178/178 [==============================] - 0s 172us/step - loss: 0.6224 - accuracy: 0.6461 - val_loss: 0.6212 - val_accuracy: 0.6494
Epoch 5/1000
178/178 [==============================] - 0s 186us/step - loss: 0.6086 - accuracy: 0.6517 - val_loss: 0.6133 - val_accuracy: 0.6623
Epoch 6/1000
178/178 [==============================] - 0s 174us/step - loss: 0.5967 - accuracy: 0.6517 - val_loss: 0.6061 - val_accuracy: 0.6623
Epoch 7/1000
178/178 [==============================] - 0s 182us/step - loss: 0.5

178/178 [==============================] - 0s 111us/step - loss: 0.4165 - accuracy: 0.8315 - val_loss: 0.5307 - val_accuracy: 0.7532
Epoch 57/1000
178/178 [==============================] - 0s 199us/step - loss: 0.4150 - accuracy: 0.8258 - val_loss: 0.5304 - val_accuracy: 0.7532
Epoch 58/1000
178/178 [==============================] - 0s 97us/step - loss: 0.4142 - accuracy: 0.8315 - val_loss: 0.5304 - val_accuracy: 0.7532
Epoch 59/1000
178/178 [==============================] - 0s 139us/step - loss: 0.4130 - accuracy: 0.8371 - val_loss: 0.5304 - val_accuracy: 0.7532
Epoch 60/1000
178/178 [==============================] - 0s 100us/step - loss: 0.4124 - accuracy: 0.8315 - val_loss: 0.5300 - val_accuracy: 0.7532
Epoch 61/1000
178/178 [==============================] - 0s 99us/step - loss: 0.4120 - accuracy: 0.8315 - val_loss: 0.5299 - val_accuracy: 0.7532
Epoch 62/1000
178/178 [==============================] - 0s 137us/step - loss: 0.4107 - accuracy: 0.8315 - val_loss: 0.5296 - val_accu

178/178 [==============================] - 0s 59us/step - loss: 0.3759 - accuracy: 0.8427 - val_loss: 0.5249 - val_accuracy: 0.7792
Epoch 113/1000
178/178 [==============================] - 0s 77us/step - loss: 0.3752 - accuracy: 0.8371 - val_loss: 0.5246 - val_accuracy: 0.7792
Epoch 114/1000
178/178 [==============================] - 0s 77us/step - loss: 0.3751 - accuracy: 0.8371 - val_loss: 0.5241 - val_accuracy: 0.7792
Epoch 115/1000
178/178 [==============================] - 0s 75us/step - loss: 0.3742 - accuracy: 0.8371 - val_loss: 0.5240 - val_accuracy: 0.7792
Epoch 116/1000
178/178 [==============================] - 0s 61us/step - loss: 0.3736 - accuracy: 0.8427 - val_loss: 0.5244 - val_accuracy: 0.7922
Epoch 117/1000
178/178 [==============================] - 0s 62us/step - loss: 0.3733 - accuracy: 0.8371 - val_loss: 0.5247 - val_accuracy: 0.7922
Epoch 118/1000
178/178 [==============================] - 0s 67us/step - loss: 0.3736 - accuracy: 0.8371 - val_loss: 0.5243 - val_acc

Epoch 168/1000
178/178 [==============================] - 0s 60us/step - loss: 0.3507 - accuracy: 0.8596 - val_loss: 0.5223 - val_accuracy: 0.7922
Epoch 169/1000
178/178 [==============================] - 0s 68us/step - loss: 0.3496 - accuracy: 0.8596 - val_loss: 0.5227 - val_accuracy: 0.7922
Epoch 170/1000
178/178 [==============================] - 0s 76us/step - loss: 0.3489 - accuracy: 0.8596 - val_loss: 0.5227 - val_accuracy: 0.7922
Epoch 171/1000
178/178 [==============================] - 0s 59us/step - loss: 0.3490 - accuracy: 0.8596 - val_loss: 0.5232 - val_accuracy: 0.7922
Epoch 172/1000
178/178 [==============================] - 0s 81us/step - loss: 0.3478 - accuracy: 0.8596 - val_loss: 0.5233 - val_accuracy: 0.7922
Epoch 173/1000
178/178 [==============================] - 0s 84us/step - loss: 0.3474 - accuracy: 0.8596 - val_loss: 0.5231 - val_accuracy: 0.7922
Epoch 174/1000
178/178 [==============================] - 0s 66us/step - loss: 0.3469 - accuracy: 0.8596 - val_loss: 0

178/178 [==============================] - 0s 80us/step - loss: 0.3286 - accuracy: 0.8820 - val_loss: 0.5205 - val_accuracy: 0.7792
Epoch 224/1000
178/178 [==============================] - 0s 69us/step - loss: 0.3272 - accuracy: 0.8764 - val_loss: 0.5199 - val_accuracy: 0.7792
Epoch 225/1000
178/178 [==============================] - 0s 72us/step - loss: 0.3264 - accuracy: 0.8708 - val_loss: 0.5198 - val_accuracy: 0.7922
Epoch 226/1000
178/178 [==============================] - 0s 67us/step - loss: 0.3256 - accuracy: 0.8820 - val_loss: 0.5199 - val_accuracy: 0.7922
Epoch 227/1000
178/178 [==============================] - 0s 77us/step - loss: 0.3258 - accuracy: 0.8764 - val_loss: 0.5195 - val_accuracy: 0.7922
Epoch 228/1000
178/178 [==============================] - 0s 67us/step - loss: 0.3259 - accuracy: 0.8820 - val_loss: 0.5200 - val_accuracy: 0.7922
Epoch 229/1000
178/178 [==============================] - 0s 67us/step - loss: 0.3247 - accuracy: 0.8820 - val_loss: 0.5197 - val_acc

...

178/178 [==============================] - 0s 112us/step - loss: 0.1221 - accuracy: 0.9663 - val_loss: 0.6185 - val_accuracy: 0.7792
Epoch 776/1000
178/178 [==============================] - 0s 149us/step - loss: 0.1208 - accuracy: 0.9663 - val_loss: 0.6162 - val_accuracy: 0.7922
Epoch 777/1000
178/178 [==============================] - 0s 104us/step - loss: 0.1211 - accuracy: 0.9663 - val_loss: 0.6183 - val_accuracy: 0.7792
Epoch 778/1000
178/178 [==============================] - 0s 142us/step - loss: 0.1203 - accuracy: 0.9663 - val_loss: 0.6215 - val_accuracy: 0.7662
Epoch 779/1000
178/178 [==============================] - 0s 132us/step - loss: 0.1219 - accuracy: 0.9663 - val_loss: 0.6259 - val_accuracy: 0.7662
Epoch 780/1000
178/178 [==============================] - 0s 104us/step - loss: 0.1213 - accuracy: 0.9663 - val_loss: 0.6214 - val_accuracy: 0.7662
Epoch 781/1000
178/178 [==============================] - 0s 107us/step - loss: 0.1198 - accuracy: 0.9663 - val_loss: 0.6209 - 

178/178 [==============================] - 0s 264us/step - loss: 0.1100 - accuracy: 0.9663 - val_loss: 0.6457 - val_accuracy: 0.7662
Epoch 831/1000
178/178 [==============================] - 0s 157us/step - loss: 0.1121 - accuracy: 0.9663 - val_loss: 0.6438 - val_accuracy: 0.7662
Epoch 832/1000
178/178 [==============================] - 0s 187us/step - loss: 0.1109 - accuracy: 0.9663 - val_loss: 0.6449 - val_accuracy: 0.7662
Epoch 833/1000
178/178 [==============================] - 0s 161us/step - loss: 0.1103 - accuracy: 0.9719 - val_loss: 0.6497 - val_accuracy: 0.7792
Epoch 834/1000
178/178 [==============================] - 0s 161us/step - loss: 0.1114 - accuracy: 0.9663 - val_loss: 0.6481 - val_accuracy: 0.7662
Epoch 835/1000
178/178 [==============================] - 0s 176us/step - loss: 0.1109 - accuracy: 0.9719 - val_loss: 0.6578 - val_accuracy: 0.7662
Epoch 836/1000
178/178 [==============================] - 0s 190us/step - loss: 0.1095 - accuracy: 0.9663 - val_loss: 0.6548 - 

Epoch 885/1000
178/178 [==============================] - 0s 80us/step - loss: 0.1009 - accuracy: 0.9719 - val_loss: 0.6760 - val_accuracy: 0.7792
Epoch 886/1000
178/178 [==============================] - 0s 84us/step - loss: 0.1012 - accuracy: 0.9663 - val_loss: 0.6743 - val_accuracy: 0.7792
Epoch 887/1000
178/178 [==============================] - 0s 82us/step - loss: 0.1002 - accuracy: 0.9719 - val_loss: 0.6707 - val_accuracy: 0.7792
Epoch 888/1000
178/178 [==============================] - 0s 79us/step - loss: 0.0999 - accuracy: 0.9663 - val_loss: 0.6701 - val_accuracy: 0.7662
Epoch 889/1000
178/178 [==============================] - 0s 66us/step - loss: 0.0993 - accuracy: 0.9719 - val_loss: 0.6720 - val_accuracy: 0.7662
Epoch 890/1000
178/178 [==============================] - 0s 73us/step - loss: 0.1012 - accuracy: 0.9719 - val_loss: 0.6760 - val_accuracy: 0.7662
Epoch 891/1000
178/178 [==============================] - 0s 73us/step - loss: 0.0990 - accuracy: 0.9719 - val_loss: 0

178/178 [==============================] - 0s 68us/step - loss: 0.0936 - accuracy: 0.9663 - val_loss: 0.6935 - val_accuracy: 0.7662
Epoch 941/1000
178/178 [==============================] - 0s 60us/step - loss: 0.0929 - accuracy: 0.9719 - val_loss: 0.6947 - val_accuracy: 0.7662
Epoch 942/1000
178/178 [==============================] - 0s 114us/step - loss: 0.0927 - accuracy: 0.9719 - val_loss: 0.6945 - val_accuracy: 0.7662
Epoch 943/1000
178/178 [==============================] - 0s 105us/step - loss: 0.0959 - accuracy: 0.9663 - val_loss: 0.6919 - val_accuracy: 0.7532
Epoch 944/1000
178/178 [==============================] - 0s 70us/step - loss: 0.0924 - accuracy: 0.9775 - val_loss: 0.7031 - val_accuracy: 0.7662
Epoch 945/1000
178/178 [==============================] - 0s 96us/step - loss: 0.0924 - accuracy: 0.9719 - val_loss: 0.6986 - val_accuracy: 0.7792
Epoch 946/1000
178/178 [==============================] - 0s 82us/step - loss: 0.0916 - accuracy: 0.9719 - val_loss: 0.7042 - val_a

178/178 [==============================] - 0s 217us/step - loss: 0.0879 - accuracy: 0.9719 - val_loss: 0.7196 - val_accuracy: 0.7532
Epoch 996/1000
178/178 [==============================] - 0s 120us/step - loss: 0.0866 - accuracy: 0.9719 - val_loss: 0.7211 - val_accuracy: 0.7662
Epoch 997/1000
178/178 [==============================] - 0s 112us/step - loss: 0.0845 - accuracy: 0.9719 - val_loss: 0.7272 - val_accuracy: 0.7792
Epoch 998/1000
178/178 [==============================] - 0s 133us/step - loss: 0.0866 - accuracy: 0.9719 - val_loss: 0.7343 - val_accuracy: 0.7532
Epoch 999/1000
178/178 [==============================] - 0s 154us/step - loss: 0.0863 - accuracy: 0.9719 - val_loss: 0.7257 - val_accuracy: 0.7792
Epoch 1000/1000
178/178 [==============================] - 0s 118us/step - loss: 0.0840 - accuracy: 0.9719 - val_loss: 0.7270 - val_accuracy: 0.7792


In [26]:
acc_test2 = model2.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test2*100))

77/77 [==============================] - 0s 148us/step
test accuracy: 77.92%


In [27]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 108), (1, 98)]


In [28]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 165), (1, 165)]


In [29]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [30]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [31]:
############# Fully-Connected Neural Network ################

In [32]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [33]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [34]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [35]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 144 samples, validate on 62 samples
Epoch 1/1000
144/144 [==============================] - 0s 1ms/step - loss: 0.5509 - accuracy: 0.8472 - val_loss: 0.5131 - val_accuracy: 0.9194
Epoch 2/1000
144/144 [==============================] - 0s 194us/step - loss: 0.4902 - accuracy: 0.9028 - val_loss: 0.4641 - val_accuracy: 0.9516
Epoch 3/1000
144/144 [==============================] - 0s 175us/step - loss: 0.4466 - accuracy: 0.9028 - val_loss: 0.4282 - val_accuracy: 0.9355
Epoch 4/1000
144/144 [==============================] - 0s 230us/step - loss: 0.4145 - accuracy: 0.8958 - val_loss: 0.3985 - val_accuracy: 0.9355
Epoch 5/1000
144/144 [==============================] - 0s 331us/step - loss: 0.3896 - accuracy: 0.8889 - val_loss: 0.3755 - val_accuracy: 0.9355
Epoch 6/1000
144/144 [==============================] - 0s 226us/step - loss: 0.3691 - accuracy: 0.8958 - val_loss: 0.3557 - val_accuracy: 0.9355
Epoch 7/1000
144/144 [==============================] - 0s 353us/step - loss: 0.3

144/144 [==============================] - 0s 237us/step - loss: 0.1832 - accuracy: 0.9167 - val_loss: 0.1734 - val_accuracy: 0.9355
Epoch 57/1000
144/144 [==============================] - 0s 263us/step - loss: 0.1812 - accuracy: 0.9167 - val_loss: 0.1723 - val_accuracy: 0.9355
Epoch 58/1000
144/144 [==============================] - 0s 659us/step - loss: 0.1803 - accuracy: 0.9167 - val_loss: 0.1715 - val_accuracy: 0.9355
Epoch 59/1000
144/144 [==============================] - 0s 299us/step - loss: 0.1792 - accuracy: 0.9167 - val_loss: 0.1712 - val_accuracy: 0.9355
Epoch 60/1000
144/144 [==============================] - 0s 608us/step - loss: 0.1790 - accuracy: 0.9167 - val_loss: 0.1715 - val_accuracy: 0.9355
Epoch 61/1000
144/144 [==============================] - 0s 352us/step - loss: 0.1773 - accuracy: 0.9167 - val_loss: 0.1699 - val_accuracy: 0.9355
Epoch 62/1000
144/144 [==============================] - 0s 165us/step - loss: 0.1763 - accuracy: 0.9167 - val_loss: 0.1687 - val_ac

Epoch 112/1000
144/144 [==============================] - 0s 100us/step - loss: 0.1334 - accuracy: 0.9444 - val_loss: 0.1446 - val_accuracy: 0.9516
Epoch 113/1000
144/144 [==============================] - 0s 124us/step - loss: 0.1328 - accuracy: 0.9583 - val_loss: 0.1433 - val_accuracy: 0.9355
Epoch 114/1000
144/144 [==============================] - 0s 128us/step - loss: 0.1325 - accuracy: 0.9444 - val_loss: 0.1428 - val_accuracy: 0.9355
Epoch 115/1000
144/144 [==============================] - 0s 187us/step - loss: 0.1313 - accuracy: 0.9444 - val_loss: 0.1421 - val_accuracy: 0.9355
Epoch 116/1000
144/144 [==============================] - 0s 246us/step - loss: 0.1311 - accuracy: 0.9444 - val_loss: 0.1415 - val_accuracy: 0.9355
Epoch 117/1000
144/144 [==============================] - 0s 181us/step - loss: 0.1304 - accuracy: 0.9444 - val_loss: 0.1413 - val_accuracy: 0.9355
Epoch 118/1000
144/144 [==============================] - 0s 205us/step - loss: 0.1295 - accuracy: 0.9444 - val_

144/144 [==============================] - 0s 165us/step - loss: 0.0999 - accuracy: 0.9653 - val_loss: 0.1261 - val_accuracy: 0.9516
Epoch 168/1000
144/144 [==============================] - 0s 224us/step - loss: 0.0992 - accuracy: 0.9653 - val_loss: 0.1251 - val_accuracy: 0.9516
Epoch 169/1000
144/144 [==============================] - 0s 278us/step - loss: 0.0993 - accuracy: 0.9653 - val_loss: 0.1252 - val_accuracy: 0.9516
Epoch 170/1000
144/144 [==============================] - 0s 138us/step - loss: 0.0985 - accuracy: 0.9653 - val_loss: 0.1243 - val_accuracy: 0.9516
Epoch 171/1000
144/144 [==============================] - 0s 144us/step - loss: 0.0980 - accuracy: 0.9653 - val_loss: 0.1246 - val_accuracy: 0.9516
Epoch 172/1000
144/144 [==============================] - 0s 144us/step - loss: 0.0979 - accuracy: 0.9653 - val_loss: 0.1246 - val_accuracy: 0.9516
Epoch 173/1000
144/144 [==============================] - 0s 186us/step - loss: 0.0970 - accuracy: 0.9653 - val_loss: 0.1243 - 

144/144 [==============================] - 0s 95us/step - loss: 0.0753 - accuracy: 0.9722 - val_loss: 0.1145 - val_accuracy: 0.9516
Epoch 223/1000
144/144 [==============================] - 0s 95us/step - loss: 0.0750 - accuracy: 0.9792 - val_loss: 0.1135 - val_accuracy: 0.9516
Epoch 224/1000
144/144 [==============================] - 0s 92us/step - loss: 0.0744 - accuracy: 0.9792 - val_loss: 0.1120 - val_accuracy: 0.9516
Epoch 225/1000
144/144 [==============================] - 0s 94us/step - loss: 0.0739 - accuracy: 0.9792 - val_loss: 0.1116 - val_accuracy: 0.9516
Epoch 226/1000
144/144 [==============================] - 0s 114us/step - loss: 0.0735 - accuracy: 0.9722 - val_loss: 0.1115 - val_accuracy: 0.9516
Epoch 227/1000
144/144 [==============================] - 0s 117us/step - loss: 0.0733 - accuracy: 0.9792 - val_loss: 0.1127 - val_accuracy: 0.9516
Epoch 228/1000
144/144 [==============================] - 0s 94us/step - loss: 0.0728 - accuracy: 0.9792 - val_loss: 0.1132 - val_a

...

144/144 [==============================] - 0s 96us/step - loss: 0.0115 - accuracy: 1.0000 - val_loss: 0.0662 - val_accuracy: 0.9839
Epoch 775/1000
144/144 [==============================] - 0s 106us/step - loss: 0.0115 - accuracy: 1.0000 - val_loss: 0.0660 - val_accuracy: 0.9839
Epoch 776/1000
144/144 [==============================] - 0s 109us/step - loss: 0.0114 - accuracy: 1.0000 - val_loss: 0.0660 - val_accuracy: 0.9839
Epoch 777/1000
144/144 [==============================] - 0s 114us/step - loss: 0.0114 - accuracy: 1.0000 - val_loss: 0.0660 - val_accuracy: 0.9839
Epoch 778/1000
144/144 [==============================] - 0s 110us/step - loss: 0.0115 - accuracy: 1.0000 - val_loss: 0.0660 - val_accuracy: 0.9839
Epoch 779/1000
144/144 [==============================] - 0s 112us/step - loss: 0.0114 - accuracy: 1.0000 - val_loss: 0.0659 - val_accuracy: 0.9839
Epoch 780/1000
144/144 [==============================] - 0s 100us/step - loss: 0.0114 - accuracy: 1.0000 - val_loss: 0.0661 - v

144/144 [==============================] - 0s 110us/step - loss: 0.0103 - accuracy: 1.0000 - val_loss: 0.0651 - val_accuracy: 0.9839
Epoch 830/1000
144/144 [==============================] - 0s 95us/step - loss: 0.0103 - accuracy: 1.0000 - val_loss: 0.0650 - val_accuracy: 0.9839
Epoch 831/1000
144/144 [==============================] - 0s 103us/step - loss: 0.0103 - accuracy: 1.0000 - val_loss: 0.0655 - val_accuracy: 0.9839
Epoch 832/1000
144/144 [==============================] - 0s 98us/step - loss: 0.0102 - accuracy: 1.0000 - val_loss: 0.0652 - val_accuracy: 0.9839
Epoch 833/1000
144/144 [==============================] - 0s 106us/step - loss: 0.0102 - accuracy: 1.0000 - val_loss: 0.0653 - val_accuracy: 0.9839
Epoch 834/1000
144/144 [==============================] - 0s 103us/step - loss: 0.0102 - accuracy: 1.0000 - val_loss: 0.0650 - val_accuracy: 0.9839
Epoch 835/1000
144/144 [==============================] - 0s 102us/step - loss: 0.0102 - accuracy: 1.0000 - val_loss: 0.0654 - va

144/144 [==============================] - 0s 87us/step - loss: 0.0093 - accuracy: 1.0000 - val_loss: 0.0641 - val_accuracy: 0.9839
Epoch 885/1000
144/144 [==============================] - 0s 105us/step - loss: 0.0092 - accuracy: 1.0000 - val_loss: 0.0641 - val_accuracy: 0.9839
Epoch 886/1000
144/144 [==============================] - 0s 93us/step - loss: 0.0093 - accuracy: 1.0000 - val_loss: 0.0639 - val_accuracy: 0.9839
Epoch 887/1000
144/144 [==============================] - ETA: 0s - loss: 0.0147 - accuracy: 1.00 - 0s 91us/step - loss: 0.0092 - accuracy: 1.0000 - val_loss: 0.0639 - val_accuracy: 0.9839
Epoch 888/1000
144/144 [==============================] - 0s 111us/step - loss: 0.0092 - accuracy: 1.0000 - val_loss: 0.0640 - val_accuracy: 0.9839
Epoch 889/1000
144/144 [==============================] - 0s 89us/step - loss: 0.0092 - accuracy: 1.0000 - val_loss: 0.0639 - val_accuracy: 0.9839
Epoch 890/1000
144/144 [==============================] - 0s 90us/step - loss: 0.0092 - a

144/144 [==============================] - 0s 96us/step - loss: 0.0082 - accuracy: 1.0000 - val_loss: 0.0626 - val_accuracy: 0.9839
Epoch 940/1000
144/144 [==============================] - 0s 96us/step - loss: 0.0083 - accuracy: 1.0000 - val_loss: 0.0624 - val_accuracy: 0.9839
Epoch 941/1000
144/144 [==============================] - 0s 89us/step - loss: 0.0083 - accuracy: 1.0000 - val_loss: 0.0624 - val_accuracy: 0.9839
Epoch 942/1000
144/144 [==============================] - 0s 92us/step - loss: 0.0082 - accuracy: 1.0000 - val_loss: 0.0625 - val_accuracy: 0.9839
Epoch 943/1000
144/144 [==============================] - 0s 94us/step - loss: 0.0082 - accuracy: 1.0000 - val_loss: 0.0624 - val_accuracy: 0.9839
Epoch 944/1000
144/144 [==============================] - 0s 99us/step - loss: 0.0082 - accuracy: 1.0000 - val_loss: 0.0624 - val_accuracy: 0.9839
Epoch 945/1000
144/144 [==============================] - 0s 96us/step - loss: 0.0082 - accuracy: 1.0000 - val_loss: 0.0623 - val_acc

144/144 [==============================] - 0s 101us/step - loss: 0.0075 - accuracy: 1.0000 - val_loss: 0.0620 - val_accuracy: 0.9839
Epoch 995/1000
144/144 [==============================] - 0s 101us/step - loss: 0.0075 - accuracy: 1.0000 - val_loss: 0.0623 - val_accuracy: 0.9839
Epoch 996/1000
144/144 [==============================] - 0s 97us/step - loss: 0.0075 - accuracy: 1.0000 - val_loss: 0.0620 - val_accuracy: 0.9839
Epoch 997/1000
144/144 [==============================] - 0s 99us/step - loss: 0.0075 - accuracy: 1.0000 - val_loss: 0.0618 - val_accuracy: 0.9839
Epoch 998/1000
144/144 [==============================] - 0s 100us/step - loss: 0.0075 - accuracy: 1.0000 - val_loss: 0.0622 - val_accuracy: 0.9839
Epoch 999/1000
144/144 [==============================] - 0s 112us/step - loss: 0.0075 - accuracy: 1.0000 - val_loss: 0.0620 - val_accuracy: 0.9839
Epoch 1000/1000
144/144 [==============================] - 0s 94us/step - loss: 0.0075 - accuracy: 1.0000 - val_loss: 0.0622 - va

In [36]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

62/62 [==============================] - 0s 68us/step
combination test accuracy: 98.39%


In [37]:
#### add regularizor and dropout
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [38]:
model1_comb2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [39]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 144 samples, validate on 62 samples
Epoch 1/1000
144/144 [==============================] - 0s 1ms/step - loss: 2.7929 - accuracy: 0.5694 - val_loss: 0.8505 - val_accuracy: 0.6774
Epoch 2/1000
144/144 [==============================] - 0s 104us/step - loss: 1.9145 - accuracy: 0.6875 - val_loss: 0.7787 - val_accuracy: 0.7742
Epoch 3/1000
144/144 [==============================] - 0s 123us/step - loss: 2.0402 - accuracy: 0.7014 - val_loss: 0.7232 - val_accuracy: 0.8226
Epoch 4/1000
144/144 [==============================] - 0s 109us/step - loss: 1.3821 - accuracy: 0.7639 - val_loss: 0.6753 - val_accuracy: 0.9194
Epoch 5/1000
144/144 [==============================] - 0s 109us/step - loss: 1.8601 - accuracy: 0.7708 - val_loss: 0.6418 - val_accuracy: 0.9355
Epoch 6/1000
144/144 [==============================] - 0s 106us/step - loss: 2.4545 - accuracy: 0.7847 - val_loss: 0.6123 - val_accuracy: 0.9355
Epoch 7/1000
144/144 [==============================] - 0s 107us/step - loss: 2.4

144/144 [==============================] - 0s 104us/step - loss: 1.6439 - accuracy: 0.8681 - val_loss: 0.3716 - val_accuracy: 0.9516
Epoch 57/1000
144/144 [==============================] - 0s 105us/step - loss: 1.4570 - accuracy: 0.8611 - val_loss: 0.3713 - val_accuracy: 0.9516
Epoch 58/1000
144/144 [==============================] - 0s 113us/step - loss: 1.7725 - accuracy: 0.8403 - val_loss: 0.3724 - val_accuracy: 0.9516
Epoch 59/1000
144/144 [==============================] - 0s 98us/step - loss: 1.4582 - accuracy: 0.8611 - val_loss: 0.3704 - val_accuracy: 0.9516
Epoch 60/1000
144/144 [==============================] - 0s 114us/step - loss: 2.5058 - accuracy: 0.7917 - val_loss: 0.3700 - val_accuracy: 0.9516
Epoch 61/1000
144/144 [==============================] - 0s 102us/step - loss: 1.7598 - accuracy: 0.8472 - val_loss: 0.3689 - val_accuracy: 0.9516
Epoch 62/1000
144/144 [==============================] - 0s 103us/step - loss: 2.6193 - accuracy: 0.7847 - val_loss: 0.3683 - val_acc

Epoch 112/1000
144/144 [==============================] - 0s 135us/step - loss: 1.3841 - accuracy: 0.8889 - val_loss: 0.3109 - val_accuracy: 0.9516
Epoch 113/1000
144/144 [==============================] - 0s 100us/step - loss: 2.3519 - accuracy: 0.8194 - val_loss: 0.3111 - val_accuracy: 0.9516
Epoch 114/1000
144/144 [==============================] - 0s 114us/step - loss: 1.6041 - accuracy: 0.8681 - val_loss: 0.3123 - val_accuracy: 0.9516
Epoch 115/1000
144/144 [==============================] - 0s 102us/step - loss: 1.5887 - accuracy: 0.8750 - val_loss: 0.3117 - val_accuracy: 0.9516
Epoch 116/1000
144/144 [==============================] - 0s 153us/step - loss: 1.9065 - accuracy: 0.8611 - val_loss: 0.3114 - val_accuracy: 0.9516
Epoch 117/1000
144/144 [==============================] - 0s 95us/step - loss: 1.4872 - accuracy: 0.8750 - val_loss: 0.3110 - val_accuracy: 0.9516
Epoch 118/1000
144/144 [==============================] - 0s 112us/step - loss: 1.1719 - accuracy: 0.9028 - val_l

144/144 [==============================] - 0s 130us/step - loss: 1.3443 - accuracy: 0.8889 - val_loss: 0.2790 - val_accuracy: 0.9839
Epoch 168/1000
144/144 [==============================] - 0s 138us/step - loss: 1.9931 - accuracy: 0.8403 - val_loss: 0.2801 - val_accuracy: 0.9839
Epoch 169/1000
144/144 [==============================] - 0s 133us/step - loss: 2.2082 - accuracy: 0.8264 - val_loss: 0.2806 - val_accuracy: 0.9839
Epoch 170/1000
144/144 [==============================] - 0s 116us/step - loss: 1.5456 - accuracy: 0.8819 - val_loss: 0.2793 - val_accuracy: 0.9839
Epoch 171/1000
144/144 [==============================] - 0s 146us/step - loss: 1.4487 - accuracy: 0.8819 - val_loss: 0.2804 - val_accuracy: 0.9839
Epoch 172/1000
144/144 [==============================] - 0s 135us/step - loss: 1.5585 - accuracy: 0.8681 - val_loss: 0.2764 - val_accuracy: 0.9839
Epoch 173/1000
144/144 [==============================] - 0s 124us/step - loss: 1.5542 - accuracy: 0.8750 - val_loss: 0.2761 - 

144/144 [==============================] - 0s 101us/step - loss: 0.8928 - accuracy: 0.9236 - val_loss: 0.2552 - val_accuracy: 0.9839
Epoch 223/1000
144/144 [==============================] - 0s 124us/step - loss: 1.8582 - accuracy: 0.8542 - val_loss: 0.2555 - val_accuracy: 0.9839
Epoch 224/1000
144/144 [==============================] - 0s 97us/step - loss: 1.5314 - accuracy: 0.8819 - val_loss: 0.2546 - val_accuracy: 0.9839
Epoch 225/1000
144/144 [==============================] - 0s 135us/step - loss: 2.0627 - accuracy: 0.8472 - val_loss: 0.2544 - val_accuracy: 0.9839
Epoch 226/1000
144/144 [==============================] - 0s 123us/step - loss: 1.6456 - accuracy: 0.8681 - val_loss: 0.2582 - val_accuracy: 0.9839
Epoch 227/1000
144/144 [==============================] - 0s 117us/step - loss: 1.4153 - accuracy: 0.8958 - val_loss: 0.2569 - val_accuracy: 0.9839
Epoch 228/1000
144/144 [==============================] - 0s 102us/step - loss: 1.8545 - accuracy: 0.8611 - val_loss: 0.2541 - v

...

144/144 [==============================] - 0s 115us/step - loss: 2.2189 - accuracy: 0.8611 - val_loss: 0.1900 - val_accuracy: 0.9839
Epoch 773/1000
144/144 [==============================] - 0s 116us/step - loss: 1.3620 - accuracy: 0.9167 - val_loss: 0.1875 - val_accuracy: 0.9839
Epoch 774/1000
144/144 [==============================] - 0s 99us/step - loss: 1.4725 - accuracy: 0.9097 - val_loss: 0.1921 - val_accuracy: 0.9839
Epoch 775/1000
144/144 [==============================] - 0s 106us/step - loss: 1.5758 - accuracy: 0.9028 - val_loss: 0.1903 - val_accuracy: 0.9839
Epoch 776/1000
144/144 [==============================] - 0s 112us/step - loss: 1.2551 - accuracy: 0.9236 - val_loss: 0.1867 - val_accuracy: 0.9839
Epoch 777/1000
144/144 [==============================] - 0s 100us/step - loss: 1.7927 - accuracy: 0.8889 - val_loss: 0.1899 - val_accuracy: 0.9839
Epoch 778/1000
144/144 [==============================] - 0s 102us/step - loss: 1.2519 - accuracy: 0.9236 - val_loss: 0.1882 - v

144/144 [==============================] - 0s 93us/step - loss: 1.1463 - accuracy: 0.9306 - val_loss: 0.1878 - val_accuracy: 0.9839
Epoch 828/1000
144/144 [==============================] - 0s 102us/step - loss: 1.4660 - accuracy: 0.9097 - val_loss: 0.1829 - val_accuracy: 0.9839
Epoch 829/1000
144/144 [==============================] - 0s 106us/step - loss: 1.7901 - accuracy: 0.8889 - val_loss: 0.1893 - val_accuracy: 0.9839
Epoch 830/1000
144/144 [==============================] - 0s 122us/step - loss: 1.0373 - accuracy: 0.9375 - val_loss: 0.1904 - val_accuracy: 0.9839
Epoch 831/1000
144/144 [==============================] - 0s 101us/step - loss: 2.1057 - accuracy: 0.8681 - val_loss: 0.1889 - val_accuracy: 0.9839
Epoch 832/1000
144/144 [==============================] - 0s 106us/step - loss: 1.1466 - accuracy: 0.9306 - val_loss: 0.1863 - val_accuracy: 0.9839
Epoch 833/1000
144/144 [==============================] - 0s 107us/step - loss: 1.0389 - accuracy: 0.9375 - val_loss: 0.1850 - v

144/144 [==============================] - 0s 103us/step - loss: 1.6791 - accuracy: 0.8958 - val_loss: 0.1860 - val_accuracy: 0.9839
Epoch 883/1000
144/144 [==============================] - 0s 118us/step - loss: 1.3569 - accuracy: 0.9167 - val_loss: 0.1860 - val_accuracy: 0.9839
Epoch 884/1000
144/144 [==============================] - 0s 101us/step - loss: 1.4630 - accuracy: 0.9097 - val_loss: 0.1868 - val_accuracy: 0.9839
Epoch 885/1000
144/144 [==============================] - 0s 121us/step - loss: 1.7845 - accuracy: 0.8889 - val_loss: 0.1842 - val_accuracy: 0.9839
Epoch 886/1000
144/144 [==============================] - 0s 104us/step - loss: 1.3544 - accuracy: 0.9167 - val_loss: 0.1883 - val_accuracy: 0.9839
Epoch 887/1000
144/144 [==============================] - 0s 102us/step - loss: 1.3559 - accuracy: 0.9167 - val_loss: 0.1851 - val_accuracy: 0.9839
Epoch 888/1000
144/144 [==============================] - 0s 113us/step - loss: 0.8215 - accuracy: 0.9514 - val_loss: 0.1840 - 

144/144 [==============================] - 0s 135us/step - loss: 1.0321 - accuracy: 0.9375 - val_loss: 0.1872 - val_accuracy: 0.9839
Epoch 938/1000
144/144 [==============================] - 0s 121us/step - loss: 2.2118 - accuracy: 0.8611 - val_loss: 0.1880 - val_accuracy: 0.9839
Epoch 939/1000
144/144 [==============================] - 0s 104us/step - loss: 1.1398 - accuracy: 0.9306 - val_loss: 0.1813 - val_accuracy: 0.9839
Epoch 940/1000
144/144 [==============================] - 0s 109us/step - loss: 1.5701 - accuracy: 0.9028 - val_loss: 0.1838 - val_accuracy: 0.9839
Epoch 941/1000
144/144 [==============================] - 0s 105us/step - loss: 1.3571 - accuracy: 0.9167 - val_loss: 0.1862 - val_accuracy: 0.9839
Epoch 942/1000
144/144 [==============================] - 0s 123us/step - loss: 1.9976 - accuracy: 0.8750 - val_loss: 0.1823 - val_accuracy: 0.9839
Epoch 943/1000
144/144 [==============================] - 0s 118us/step - loss: 1.3517 - accuracy: 0.9167 - val_loss: 0.1850 - 

144/144 [==============================] - 0s 106us/step - loss: 1.8886 - accuracy: 0.8819 - val_loss: 0.1871 - val_accuracy: 0.9839
Epoch 993/1000
144/144 [==============================] - 0s 110us/step - loss: 1.9943 - accuracy: 0.8750 - val_loss: 0.1838 - val_accuracy: 0.9839
Epoch 994/1000
144/144 [==============================] - 0s 123us/step - loss: 1.3548 - accuracy: 0.9167 - val_loss: 0.1865 - val_accuracy: 0.9839
Epoch 995/1000
144/144 [==============================] - 0s 105us/step - loss: 0.8192 - accuracy: 0.9514 - val_loss: 0.1846 - val_accuracy: 0.9839
Epoch 996/1000
144/144 [==============================] - 0s 108us/step - loss: 1.4576 - accuracy: 0.9097 - val_loss: 0.1872 - val_accuracy: 0.9839
Epoch 997/1000
144/144 [==============================] - 0s 130us/step - loss: 1.9939 - accuracy: 0.8750 - val_loss: 0.1821 - val_accuracy: 0.9839
Epoch 998/1000
144/144 [==============================] - 0s 109us/step - loss: 1.4574 - accuracy: 0.9097 - val_loss: 0.1861 - 

In [40]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

62/62 [==============================] - 0s 100us/step
combination test accuracy: 98.39%


In [41]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [42]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [43]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 231 samples, validate on 99 samples
Epoch 1/1000
231/231 [==============================] - 0s 458us/step - loss: 0.7233 - accuracy: 0.4675 - val_loss: 0.7008 - val_accuracy: 0.4848
Epoch 2/1000
231/231 [==============================] - 0s 85us/step - loss: 0.6487 - accuracy: 0.5974 - val_loss: 0.6621 - val_accuracy: 0.5960
Epoch 3/1000
231/231 [==============================] - 0s 92us/step - loss: 0.6163 - accuracy: 0.6407 - val_loss: 0.6355 - val_accuracy: 0.6566
Epoch 4/1000
231/231 [==============================] - 0s 91us/step - loss: 0.5914 - accuracy: 0.6970 - val_loss: 0.6152 - val_accuracy: 0.7273
Epoch 5/1000
231/231 [==============================] - 0s 89us/step - loss: 0.5728 - accuracy: 0.7619 - val_loss: 0.5982 - val_accuracy: 0.7475
Epoch 6/1000
231/231 [==============================] - 0s 126us/step - loss: 0.5557 - accuracy: 0.7706 - val_loss: 0.5817 - val_accuracy: 0.7879
Epoch 7/1000
231/231 [==============================] - 0s 116us/step - loss: 0.542

231/231 [==============================] - 0s 82us/step - loss: 0.4026 - accuracy: 0.8052 - val_loss: 0.4738 - val_accuracy: 0.7576
Epoch 57/1000
231/231 [==============================] - 0s 86us/step - loss: 0.4028 - accuracy: 0.8009 - val_loss: 0.4737 - val_accuracy: 0.7778
Epoch 58/1000
231/231 [==============================] - 0s 79us/step - loss: 0.4016 - accuracy: 0.8139 - val_loss: 0.4675 - val_accuracy: 0.7980
Epoch 59/1000
231/231 [==============================] - 0s 82us/step - loss: 0.3987 - accuracy: 0.8139 - val_loss: 0.4672 - val_accuracy: 0.7879
Epoch 60/1000
231/231 [==============================] - 0s 79us/step - loss: 0.3967 - accuracy: 0.8139 - val_loss: 0.4664 - val_accuracy: 0.8081
Epoch 61/1000
231/231 [==============================] - 0s 80us/step - loss: 0.3975 - accuracy: 0.8312 - val_loss: 0.4645 - val_accuracy: 0.7879
Epoch 62/1000
231/231 [==============================] - 0s 82us/step - loss: 0.3976 - accuracy: 0.8052 - val_loss: 0.4644 - val_accuracy:

231/231 [==============================] - 0s 83us/step - loss: 0.3413 - accuracy: 0.8658 - val_loss: 0.4472 - val_accuracy: 0.8384
Epoch 113/1000
231/231 [==============================] - 0s 84us/step - loss: 0.3380 - accuracy: 0.8615 - val_loss: 0.4474 - val_accuracy: 0.8384
Epoch 114/1000
231/231 [==============================] - 0s 84us/step - loss: 0.3381 - accuracy: 0.8615 - val_loss: 0.4527 - val_accuracy: 0.8485
Epoch 115/1000
231/231 [==============================] - 0s 83us/step - loss: 0.3379 - accuracy: 0.8701 - val_loss: 0.4465 - val_accuracy: 0.8384
Epoch 116/1000
231/231 [==============================] - 0s 82us/step - loss: 0.3337 - accuracy: 0.8615 - val_loss: 0.4529 - val_accuracy: 0.8081
Epoch 117/1000
231/231 [==============================] - 0s 95us/step - loss: 0.3368 - accuracy: 0.8701 - val_loss: 0.4468 - val_accuracy: 0.8283
Epoch 118/1000
231/231 [==============================] - 0s 74us/step - loss: 0.3317 - accuracy: 0.8658 - val_loss: 0.4521 - val_acc

Epoch 168/1000
231/231 [==============================] - 0s 81us/step - loss: 0.2865 - accuracy: 0.8831 - val_loss: 0.4349 - val_accuracy: 0.8687
Epoch 169/1000
231/231 [==============================] - 0s 84us/step - loss: 0.2867 - accuracy: 0.8961 - val_loss: 0.4414 - val_accuracy: 0.8687
Epoch 170/1000
231/231 [==============================] - 0s 82us/step - loss: 0.2860 - accuracy: 0.8831 - val_loss: 0.4345 - val_accuracy: 0.8586
Epoch 171/1000
231/231 [==============================] - 0s 82us/step - loss: 0.2860 - accuracy: 0.8918 - val_loss: 0.4431 - val_accuracy: 0.8687
Epoch 172/1000
231/231 [==============================] - 0s 80us/step - loss: 0.2875 - accuracy: 0.8831 - val_loss: 0.4382 - val_accuracy: 0.8687
Epoch 173/1000
231/231 [==============================] - 0s 92us/step - loss: 0.2834 - accuracy: 0.8918 - val_loss: 0.4518 - val_accuracy: 0.8687
Epoch 174/1000
231/231 [==============================] - 0s 80us/step - loss: 0.2842 - accuracy: 0.8918 - val_loss: 0

Epoch 224/1000
231/231 [==============================] - 0s 83us/step - loss: 0.2483 - accuracy: 0.9177 - val_loss: 0.4446 - val_accuracy: 0.8485
Epoch 225/1000
231/231 [==============================] - 0s 83us/step - loss: 0.2512 - accuracy: 0.9048 - val_loss: 0.4406 - val_accuracy: 0.8586
Epoch 226/1000
231/231 [==============================] - 0s 82us/step - loss: 0.2504 - accuracy: 0.9221 - val_loss: 0.4383 - val_accuracy: 0.8687
Epoch 227/1000
231/231 [==============================] - 0s 87us/step - loss: 0.2442 - accuracy: 0.9221 - val_loss: 0.4372 - val_accuracy: 0.8687
Epoch 228/1000
231/231 [==============================] - 0s 81us/step - loss: 0.2454 - accuracy: 0.9134 - val_loss: 0.4334 - val_accuracy: 0.8687
Epoch 229/1000
231/231 [==============================] - 0s 80us/step - loss: 0.2438 - accuracy: 0.9221 - val_loss: 0.4350 - val_accuracy: 0.8687
Epoch 230/1000
231/231 [==============================] - 0s 79us/step - loss: 0.2460 - accuracy: 0.9134 - val_loss: 0

...

Epoch 727/1000
231/231 [==============================] - 0s 81us/step - loss: 0.1114 - accuracy: 0.9697 - val_loss: 0.4921 - val_accuracy: 0.8485
Epoch 728/1000
231/231 [==============================] - 0s 75us/step - loss: 0.1095 - accuracy: 0.9697 - val_loss: 0.5172 - val_accuracy: 0.8687
Epoch 729/1000
231/231 [==============================] - 0s 85us/step - loss: 0.1076 - accuracy: 0.9697 - val_loss: 0.6015 - val_accuracy: 0.8081
Epoch 730/1000
231/231 [==============================] - 0s 80us/step - loss: 0.1114 - accuracy: 0.9567 - val_loss: 0.4876 - val_accuracy: 0.8485
Epoch 731/1000
231/231 [==============================] - 0s 81us/step - loss: 0.1098 - accuracy: 0.9697 - val_loss: 0.5487 - val_accuracy: 0.8283
Epoch 732/1000
231/231 [==============================] - 0s 78us/step - loss: 0.1091 - accuracy: 0.9697 - val_loss: 0.5265 - val_accuracy: 0.8687
Epoch 733/1000
231/231 [==============================] - 0s 81us/step - loss: 0.1077 - accuracy: 0.9697 - val_loss: 0

Epoch 783/1000
231/231 [==============================] - 0s 85us/step - loss: 0.1082 - accuracy: 0.9610 - val_loss: 0.5173 - val_accuracy: 0.8485
Epoch 784/1000
231/231 [==============================] - 0s 81us/step - loss: 0.1056 - accuracy: 0.9697 - val_loss: 0.4988 - val_accuracy: 0.8485
Epoch 785/1000
231/231 [==============================] - 0s 85us/step - loss: 0.1043 - accuracy: 0.9697 - val_loss: 0.5274 - val_accuracy: 0.8687
Epoch 786/1000
231/231 [==============================] - 0s 83us/step - loss: 0.1019 - accuracy: 0.9740 - val_loss: 0.5226 - val_accuracy: 0.8687
Epoch 787/1000
231/231 [==============================] - 0s 84us/step - loss: 0.1034 - accuracy: 0.9697 - val_loss: 0.4905 - val_accuracy: 0.8586
Epoch 788/1000
231/231 [==============================] - 0s 83us/step - loss: 0.1089 - accuracy: 0.9654 - val_loss: 0.4957 - val_accuracy: 0.8586
Epoch 789/1000
231/231 [==============================] - 0s 84us/step - loss: 0.1036 - accuracy: 0.9654 - val_loss: 0

Epoch 839/1000
231/231 [==============================] - 0s 81us/step - loss: 0.0988 - accuracy: 0.9654 - val_loss: 0.5477 - val_accuracy: 0.8586
Epoch 840/1000
231/231 [==============================] - 0s 78us/step - loss: 0.0970 - accuracy: 0.9697 - val_loss: 0.5232 - val_accuracy: 0.8687
Epoch 841/1000
231/231 [==============================] - 0s 83us/step - loss: 0.0987 - accuracy: 0.9610 - val_loss: 0.5060 - val_accuracy: 0.8485
Epoch 842/1000
231/231 [==============================] - 0s 82us/step - loss: 0.0971 - accuracy: 0.9697 - val_loss: 0.5337 - val_accuracy: 0.8687
Epoch 843/1000
231/231 [==============================] - 0s 85us/step - loss: 0.0970 - accuracy: 0.9697 - val_loss: 0.5000 - val_accuracy: 0.8586
Epoch 844/1000
231/231 [==============================] - 0s 83us/step - loss: 0.0998 - accuracy: 0.9697 - val_loss: 0.5091 - val_accuracy: 0.8485
Epoch 845/1000
231/231 [==============================] - 0s 93us/step - loss: 0.0965 - accuracy: 0.9697 - val_loss: 0

Epoch 895/1000
231/231 [==============================] - 0s 87us/step - loss: 0.0928 - accuracy: 0.9697 - val_loss: 0.5205 - val_accuracy: 0.8485
Epoch 896/1000
231/231 [==============================] - 0s 81us/step - loss: 0.0945 - accuracy: 0.9697 - val_loss: 0.5480 - val_accuracy: 0.8687
Epoch 897/1000
231/231 [==============================] - 0s 85us/step - loss: 0.0946 - accuracy: 0.9697 - val_loss: 0.5379 - val_accuracy: 0.8687
Epoch 898/1000
231/231 [==============================] - 0s 83us/step - loss: 0.0935 - accuracy: 0.9697 - val_loss: 0.5410 - val_accuracy: 0.8687
Epoch 899/1000
231/231 [==============================] - 0s 85us/step - loss: 0.0936 - accuracy: 0.9697 - val_loss: 0.5136 - val_accuracy: 0.8485
Epoch 900/1000
231/231 [==============================] - 0s 81us/step - loss: 0.0944 - accuracy: 0.9697 - val_loss: 0.5157 - val_accuracy: 0.8586
Epoch 901/1000
231/231 [==============================] - 0s 81us/step - loss: 0.0957 - accuracy: 0.9654 - val_loss: 0

Epoch 951/1000
231/231 [==============================] - 0s 85us/step - loss: 0.0948 - accuracy: 0.9654 - val_loss: 0.5503 - val_accuracy: 0.8586
Epoch 952/1000
231/231 [==============================] - 0s 80us/step - loss: 0.0906 - accuracy: 0.9697 - val_loss: 0.5364 - val_accuracy: 0.8485
Epoch 953/1000
231/231 [==============================] - 0s 82us/step - loss: 0.0914 - accuracy: 0.9697 - val_loss: 0.5259 - val_accuracy: 0.8485
Epoch 954/1000
231/231 [==============================] - 0s 88us/step - loss: 0.0926 - accuracy: 0.9697 - val_loss: 0.5594 - val_accuracy: 0.8586
Epoch 955/1000
231/231 [==============================] - 0s 81us/step - loss: 0.0928 - accuracy: 0.9697 - val_loss: 0.5518 - val_accuracy: 0.8586
Epoch 956/1000
231/231 [==============================] - 0s 87us/step - loss: 0.0926 - accuracy: 0.9697 - val_loss: 0.6674 - val_accuracy: 0.8081
Epoch 957/1000
231/231 [==============================] - 0s 82us/step - loss: 0.0988 - accuracy: 0.9567 - val_loss: 0

In [44]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

99/99 [==============================] - 0s 77us/step
over-sampling test accuracy: 84.85%


In [45]:
#### add regularizer and dropout on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [46]:
model1_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [47]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 231 samples, validate on 99 samples
Epoch 1/1000
231/231 [==============================] - 0s 707us/step - loss: 2.2556 - accuracy: 0.4805 - val_loss: 0.8995 - val_accuracy: 0.6465
Epoch 2/1000
231/231 [==============================] - 0s 88us/step - loss: 2.5999 - accuracy: 0.5714 - val_loss: 0.8352 - val_accuracy: 0.6768
Epoch 3/1000
231/231 [==============================] - 0s 90us/step - loss: 2.1134 - accuracy: 0.6710 - val_loss: 0.7791 - val_accuracy: 0.7273
Epoch 4/1000
231/231 [==============================] - 0s 92us/step - loss: 2.1087 - accuracy: 0.6970 - val_loss: 0.7406 - val_accuracy: 0.7475
Epoch 5/1000
231/231 [==============================] - 0s 98us/step - loss: 2.1509 - accuracy: 0.6840 - val_loss: 0.7161 - val_accuracy: 0.7273
Epoch 6/1000
231/231 [==============================] - 0s 92us/step - loss: 2.1753 - accuracy: 0.7100 - val_loss: 0.6950 - val_accuracy: 0.7778
Epoch 7/1000
231/231 [==============================] - 0s 96us/step - loss: 2.2249 

Epoch 57/1000
231/231 [==============================] - 0s 86us/step - loss: 2.0104 - accuracy: 0.7359 - val_loss: 0.5778 - val_accuracy: 0.8384
Epoch 58/1000
231/231 [==============================] - 0s 96us/step - loss: 2.0919 - accuracy: 0.7273 - val_loss: 0.5761 - val_accuracy: 0.8384
Epoch 59/1000
231/231 [==============================] - 0s 87us/step - loss: 1.5603 - accuracy: 0.7792 - val_loss: 0.5819 - val_accuracy: 0.8384
Epoch 60/1000
231/231 [==============================] - 0s 91us/step - loss: 1.5750 - accuracy: 0.7532 - val_loss: 0.5809 - val_accuracy: 0.8283
Epoch 61/1000
231/231 [==============================] - 0s 100us/step - loss: 3.1140 - accuracy: 0.6840 - val_loss: 0.5759 - val_accuracy: 0.8384
Epoch 62/1000
231/231 [==============================] - 0s 100us/step - loss: 2.4820 - accuracy: 0.6883 - val_loss: 0.5721 - val_accuracy: 0.8485
Epoch 63/1000
231/231 [==============================] - 0s 90us/step - loss: 2.0799 - accuracy: 0.7403 - val_loss: 0.5696

Epoch 113/1000
231/231 [==============================] - 0s 89us/step - loss: 1.5754 - accuracy: 0.7835 - val_loss: 0.5420 - val_accuracy: 0.8586
Epoch 114/1000
231/231 [==============================] - 0s 84us/step - loss: 1.5188 - accuracy: 0.7749 - val_loss: 0.5457 - val_accuracy: 0.8586
Epoch 115/1000
231/231 [==============================] - 0s 90us/step - loss: 1.6148 - accuracy: 0.7965 - val_loss: 0.5499 - val_accuracy: 0.8485
Epoch 116/1000
231/231 [==============================] - 0s 89us/step - loss: 2.2841 - accuracy: 0.7403 - val_loss: 0.5414 - val_accuracy: 0.8586
Epoch 117/1000
231/231 [==============================] - 0s 85us/step - loss: 2.6075 - accuracy: 0.7273 - val_loss: 0.5448 - val_accuracy: 0.8586
Epoch 118/1000
231/231 [==============================] - 0s 87us/step - loss: 2.0385 - accuracy: 0.7619 - val_loss: 0.5474 - val_accuracy: 0.8586
Epoch 119/1000
231/231 [==============================] - 0s 80us/step - loss: 1.8172 - accuracy: 0.7922 - val_loss: 0

231/231 [==============================] - 0s 96us/step - loss: 2.0111 - accuracy: 0.7922 - val_loss: 0.5296 - val_accuracy: 0.8586
Epoch 169/1000
231/231 [==============================] - 0s 101us/step - loss: 2.0026 - accuracy: 0.8009 - val_loss: 0.5314 - val_accuracy: 0.8586
Epoch 170/1000
231/231 [==============================] - 0s 92us/step - loss: 2.2711 - accuracy: 0.7879 - val_loss: 0.5309 - val_accuracy: 0.8586
Epoch 171/1000
231/231 [==============================] - 0s 113us/step - loss: 1.7818 - accuracy: 0.8268 - val_loss: 0.5275 - val_accuracy: 0.8586
Epoch 172/1000
231/231 [==============================] - 0s 99us/step - loss: 1.8323 - accuracy: 0.8312 - val_loss: 0.5245 - val_accuracy: 0.8586
Epoch 173/1000
231/231 [==============================] - 0s 105us/step - loss: 1.7201 - accuracy: 0.8355 - val_loss: 0.5307 - val_accuracy: 0.8485
Epoch 174/1000
231/231 [==============================] - 0s 98us/step - loss: 1.5585 - accuracy: 0.8528 - val_loss: 0.5282 - val_

Epoch 224/1000
231/231 [==============================] - 0s 88us/step - loss: 1.6887 - accuracy: 0.8485 - val_loss: 0.5141 - val_accuracy: 0.8586
Epoch 225/1000
231/231 [==============================] - 0s 87us/step - loss: 1.8221 - accuracy: 0.8312 - val_loss: 0.5203 - val_accuracy: 0.8586
Epoch 226/1000
231/231 [==============================] - 0s 88us/step - loss: 1.4115 - accuracy: 0.8745 - val_loss: 0.5289 - val_accuracy: 0.8485
Epoch 227/1000
231/231 [==============================] - 0s 87us/step - loss: 2.0831 - accuracy: 0.8225 - val_loss: 0.5380 - val_accuracy: 0.8384
Epoch 228/1000
231/231 [==============================] - 0s 101us/step - loss: 1.5512 - accuracy: 0.8571 - val_loss: 0.5201 - val_accuracy: 0.8586
Epoch 229/1000
231/231 [==============================] - 0s 88us/step - loss: 2.0828 - accuracy: 0.8225 - val_loss: 0.5104 - val_accuracy: 0.8586
Epoch 230/1000
231/231 [==============================] - 0s 92us/step - loss: 1.8092 - accuracy: 0.8398 - val_loss: 

...

231/231 [==============================] - 0s 89us/step - loss: 1.8151 - accuracy: 0.8658 - val_loss: 0.5669 - val_accuracy: 0.8586
Epoch 727/1000
231/231 [==============================] - 0s 86us/step - loss: 1.4961 - accuracy: 0.8788 - val_loss: 0.5566 - val_accuracy: 0.8687
Epoch 728/1000
231/231 [==============================] - 0s 86us/step - loss: 2.2128 - accuracy: 0.8398 - val_loss: 0.5383 - val_accuracy: 0.8586
Epoch 729/1000
231/231 [==============================] - 0s 85us/step - loss: 1.8210 - accuracy: 0.8615 - val_loss: 0.5606 - val_accuracy: 0.8485
Epoch 730/1000
231/231 [==============================] - 0s 94us/step - loss: 1.9454 - accuracy: 0.8571 - val_loss: 0.5524 - val_accuracy: 0.8687
Epoch 731/1000
231/231 [==============================] - 0s 93us/step - loss: 1.7510 - accuracy: 0.8658 - val_loss: 0.5610 - val_accuracy: 0.8586
Epoch 732/1000
231/231 [==============================] - 0s 93us/step - loss: 2.0118 - accuracy: 0.8528 - val_loss: 0.5502 - val_acc

231/231 [==============================] - 0s 94us/step - loss: 1.8104 - accuracy: 0.8658 - val_loss: 0.5616 - val_accuracy: 0.8687
Epoch 782/1000
231/231 [==============================] - 0s 90us/step - loss: 1.8086 - accuracy: 0.8658 - val_loss: 0.5656 - val_accuracy: 0.8687
Epoch 783/1000
231/231 [==============================] - 0s 91us/step - loss: 1.6018 - accuracy: 0.8788 - val_loss: 0.5597 - val_accuracy: 0.8687
Epoch 784/1000
231/231 [==============================] - 0s 89us/step - loss: 1.5553 - accuracy: 0.8745 - val_loss: 0.5841 - val_accuracy: 0.8586
Epoch 785/1000
231/231 [==============================] - 0s 112us/step - loss: 1.8870 - accuracy: 0.8528 - val_loss: 0.5686 - val_accuracy: 0.8586
Epoch 786/1000
231/231 [==============================] - 0s 90us/step - loss: 2.0852 - accuracy: 0.8355 - val_loss: 0.6264 - val_accuracy: 0.8182
Epoch 787/1000
231/231 [==============================] - 0s 97us/step - loss: 1.8303 - accuracy: 0.8571 - val_loss: 0.6289 - val_ac

Epoch 837/1000
231/231 [==============================] - 0s 89us/step - loss: 1.9441 - accuracy: 0.8571 - val_loss: 0.5738 - val_accuracy: 0.8485
Epoch 838/1000
231/231 [==============================] - 0s 91us/step - loss: 1.8669 - accuracy: 0.8615 - val_loss: 0.5729 - val_accuracy: 0.8485
Epoch 839/1000
231/231 [==============================] - 0s 83us/step - loss: 1.5974 - accuracy: 0.8874 - val_loss: 0.5585 - val_accuracy: 0.8485
Epoch 840/1000
231/231 [==============================] - 0s 96us/step - loss: 1.4847 - accuracy: 0.8831 - val_loss: 0.5834 - val_accuracy: 0.8485
Epoch 841/1000
231/231 [==============================] - 0s 87us/step - loss: 2.0167 - accuracy: 0.8485 - val_loss: 0.5565 - val_accuracy: 0.8485
Epoch 842/1000
231/231 [==============================] - 0s 92us/step - loss: 1.4838 - accuracy: 0.8831 - val_loss: 0.5849 - val_accuracy: 0.8586
Epoch 843/1000
231/231 [==============================] - 0s 91us/step - loss: 1.3482 - accuracy: 0.8874 - val_loss: 0

Epoch 893/1000
231/231 [==============================] - 0s 95us/step - loss: 1.5384 - accuracy: 0.8831 - val_loss: 0.5897 - val_accuracy: 0.8384
Epoch 894/1000
231/231 [==============================] - 0s 87us/step - loss: 1.9493 - accuracy: 0.8528 - val_loss: 0.5610 - val_accuracy: 0.8586
Epoch 895/1000
231/231 [==============================] - 0s 83us/step - loss: 1.6789 - accuracy: 0.8701 - val_loss: 0.5750 - val_accuracy: 0.8384
Epoch 896/1000
231/231 [==============================] - 0s 96us/step - loss: 1.5447 - accuracy: 0.8745 - val_loss: 0.5831 - val_accuracy: 0.8485
Epoch 897/1000
231/231 [==============================] - 0s 102us/step - loss: 1.6056 - accuracy: 0.8745 - val_loss: 0.5955 - val_accuracy: 0.8283
Epoch 898/1000
231/231 [==============================] - 0s 86us/step - loss: 1.2744 - accuracy: 0.9004 - val_loss: 0.6073 - val_accuracy: 0.8485
Epoch 899/1000
231/231 [==============================] - 0s 87us/step - loss: 2.1444 - accuracy: 0.8398 - val_loss: 

Epoch 949/1000
231/231 [==============================] - 0s 90us/step - loss: 1.9238 - accuracy: 0.8615 - val_loss: 0.5883 - val_accuracy: 0.8485
Epoch 950/1000
231/231 [==============================] - 0s 96us/step - loss: 1.5937 - accuracy: 0.8788 - val_loss: 0.5711 - val_accuracy: 0.8485
Epoch 951/1000
231/231 [==============================] - 0s 89us/step - loss: 1.7869 - accuracy: 0.8701 - val_loss: 0.5937 - val_accuracy: 0.8485
Epoch 952/1000
231/231 [==============================] - 0s 103us/step - loss: 1.5892 - accuracy: 0.8874 - val_loss: 0.5926 - val_accuracy: 0.8485
Epoch 953/1000
231/231 [==============================] - 0s 83us/step - loss: 1.4720 - accuracy: 0.8831 - val_loss: 0.5781 - val_accuracy: 0.8586
Epoch 954/1000
231/231 [==============================] - 0s 96us/step - loss: 1.9385 - accuracy: 0.8528 - val_loss: 0.5732 - val_accuracy: 0.8485
Epoch 955/1000
231/231 [==============================] - 0s 89us/step - loss: 2.1150 - accuracy: 0.8528 - val_loss: 

In [48]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

99/99 [==============================] - 0s 104us/step
over-sampling test accuracy: 84.85%


In [49]:
############ Logistic Regression ############

In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [51]:
###### logistics
log = LogisticRegression(random_state=123)
log.fit(X_train, y_train)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [52]:
y_pred = log.predict(X_test)
print('logistic test accuracy %.2f%%' % (log.score(X_test, y_test)*100))

logistic test accuracy 77.92%


In [53]:
###### logistics on combination data
log_comb = LogisticRegression(random_state=123)
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [54]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 98.39%


In [55]:
##### logistics on over-sampling data
log_over = LogisticRegression(random_state=123)
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [56]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 86.87%


In [57]:
############## Random Forest ##############

In [58]:
###### random forest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=123)
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [59]:
from sklearn.metrics import accuracy_score
y_pred = rf.predict(X_test)
print('test accuracy: %.2f%%' % (accuracy_score(y_test, y_pred)*100))

test accuracy: 77.92%


In [60]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100, random_state=123)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [61]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 98.39%


In [62]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100, random_state=123)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [63]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 87.88%


In [64]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [65]:
## random forest model with CV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv = RandomForestClassifier(n_estimators=100, random_state=123)

accs = cross_val_score(estimator=rf, X=X_train, y=y_train, cv=5)
print(accs)
print(accs.mean())

[0.80555556 0.88888889 0.83333333 0.85714286 0.8       ]
0.836984126984127


In [66]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rf_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[1.         1.         0.93103448 0.93103448 1.        ]
0.9724137931034482


In [67]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.89361702 0.84782609 0.82608696 0.73913043 0.89130435]
0.8395929694727104
